# Timer

In [4]:
from datetime import datetime as dt

start_time = "25.11.2022, 11:15"
start_time = dt.strptime(start_time, "%d.%m.%Y, %H:%M")

while True:
    if start_time < dt.now():
        break
    else:
        sleep(10)

# Parameter optimization

In [5]:
import sys
import pandas as pd
import numpy as np
from glob import glob
from optimizer import Optimizer
from os import environ

# Set environment variable
environ["ENV"] = "optimize"

from bot.bot import SigBot
from config.config import ConfigFactory

pd.set_option('display.max_columns', 500)

ttype = 'buy'
pattern = ['PriceChange', 'LinearReg']
work_timeframe = '5m'
higher_timeframe = '1h'
opt_limit = 100
load = False

print(f'Timeframe is {work_timeframe}/{higher_timeframe}, trade type is {ttype}')

# Get configs
configs = ConfigFactory.factory(environ).configs
configs['Timeframes']['work_timeframe'] = work_timeframe
configs['Timeframes']['higher_timeframe'] = higher_timeframe

optim_dict = {
                'PriceChange': {"low_price_quantile": [1 * (i +  1) for i in range(20)]}, 
                'LinearReg': {'timeperiod': [6, 8, 10], 'low_bound': [0]}
             }

opt = Optimizer(pattern, optim_dict, **configs)
stat = opt.optimize(pattern, ttype, opt_limit, load)

stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
if not stat_list:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}.pkl')
else:
    stat.to_pickle(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}_{len(stat_list)}.pkl')

Timeframe is 5m/1h, trade type is buy
Number of combinations is 60


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 60/60 [05:11<00:00,  5.19s/it]


# Check local statistics

In [7]:
stat[stat['PriceChange_low_price_quantile'] == 10]

,pattern,PriceChange_low_price_quantile,LinearReg_timeperiod,LinearReg_low_bound,pct_right_forecast_1,pct_right_forecast_2,pct_right_forecast_3,pct_right_forecast_4,pct_right_forecast_5,pct_right_forecast_6,pct_right_forecast_7,pct_right_forecast_8,pct_right_forecast_9,pct_right_forecast_10,pct_right_forecast_11,pct_right_forecast_12,pct_right_forecast_13,pct_right_forecast_14,pct_right_forecast_15,pct_right_forecast_16,pct_right_forecast_17,pct_right_forecast_18,pct_right_forecast_19,pct_right_forecast_20,pct_right_forecast_21,pct_right_forecast_22,pct_right_forecast_23,pct_right_forecast_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,pct_right_forecast_avg,pct_price_diff_avg,forecast_rank,price_rank
27,PriceChange_LinearReg,10,6,0,91.60,73.28,65.65,67.94,61.83,59.54,58.78,55.73,53.44,51.91,48.85,49.62,48.09,48.85,45.80,44.27,42.75,40.46,35.88,37.40,40.46,38.93,38.93,43.51,0.55,0.62,0.51,0.67,0.42,0.49,0.43,0.32,0.18,0.12,0.00,0.00,0.00,-0.04,-0.28,-0.12,-0.42,-0.58,-0.65,-0.54,-0.60,-0.76,-0.54,-0.51,131,51.81250,-0.030417,-2382.56250,-3.984583
28,PriceChange_LinearReg,10,8,0,89.73,72.60,64.38,66.44,61.64,60.96,60.96,56.16,54.11,54.11,50.68,51.37,50.00,50.68,47.26,45.89,44.52,42.47,38.36,39.73,42.47,41.10,41.78,45.89,0.50,0.60,0.45,0.58,0.46,0.47,0.46,0.33,0.22,0.22,0.12,0.09,0.03,0.08,-0.12,-0.06,-0.32,-0.43,-0.58,-0.53,-0.50,-0.71,-0.44,-0.41,146,53.05375,0.021250,-2474.15250,3.102500
29,PriceChange_LinearReg,10,10,0,89.68,74.19,65.16,65.16,61.29,60.65,59.35,53.55,50.32,50.97,47.74,49.03,48.39,48.39,46.45,45.81,44.52,41.94,38.06,39.35,41.94,40.00,41.29,45.16,0.50,0.51,0.42,0.54,0.40,0.42,0.43,0.22,0.02,0.12,0.00,0.00,0.00,0.00,-0.10,-0.12,-0.31,-0.41,-0.50,-0.50,-0.47,-0.59,-0.45,-0.43,155,52.01625,-0.012500,-2787.48125,-1.937500


In [6]:
# 5m/1h
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
stat.sort_values('pct_right_forecast_avg', ascending=False).head(20)

,pattern,PriceChange_low_price_quantile,LinearReg_timeperiod,LinearReg_low_bound,pct_right_forecast_1,pct_right_forecast_2,pct_right_forecast_3,pct_right_forecast_4,pct_right_forecast_5,pct_right_forecast_6,pct_right_forecast_7,pct_right_forecast_8,pct_right_forecast_9,pct_right_forecast_10,pct_right_forecast_11,pct_right_forecast_12,pct_right_forecast_13,pct_right_forecast_14,pct_right_forecast_15,pct_right_forecast_16,pct_right_forecast_17,pct_right_forecast_18,pct_right_forecast_19,pct_right_forecast_20,pct_right_forecast_21,pct_right_forecast_22,pct_right_forecast_23,pct_right_forecast_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,pct_right_forecast_avg,pct_price_diff_avg,forecast_rank,price_rank
49,PriceChange_LinearReg,17,8,0,90.08,74.79,66.94,67.77,64.88,63.64,66.12,62.40,57.85,57.02,54.13,54.96,52.48,54.13,51.65,50.41,49.59,49.59,45.87,44.21,46.28,47.11,46.28,50.00,0.50,0.56,0.50,0.59,0.52,0.45,0.46,0.42,0.43,0.37,0.27,0.22,0.12,0.30,0.15,0.03,0.00,0.00,-0.13,-0.12,-0.15,-0.23,-0.24,0.03,242,57.007500,0.210417,-3144.185000,50.920833
37,PriceChange_LinearReg,13,8,0,91.40,75.81,67.74,69.89,64.52,63.98,65.59,60.22,57.53,56.99,54.30,54.84,52.69,54.84,51.08,48.92,48.39,48.92,45.16,44.62,46.24,47.31,45.70,50.54,0.55,0.63,0.61,0.67,0.56,0.51,0.53,0.43,0.43,0.41,0.27,0.26,0.22,0.31,0.15,0.00,-0.07,-0.10,-0.15,-0.14,-0.09,-0.23,-0.19,0.08,186,56.967500,0.235417,-2424.045000,43.787500
55,PriceChange_LinearReg,19,8,0,90.08,74.43,66.41,67.56,64.89,63.74,66.03,62.21,57.63,56.87,54.20,54.58,53.44,54.58,52.29,50.38,49.62,49.62,45.80,43.89,45.80,46.56,45.04,48.09,0.49,0.52,0.43,0.57,0.51,0.44,0.46,0.39,0.40,0.33,0.23,0.17,0.15,0.26,0.16,0.03,0.00,0.00,-0.15,-0.12,-0.23,-0.31,-0.26,-0.16,262,56.822500,0.179583,-3452.505000,47.050833
34,PriceChange_LinearReg,12,8,0,91.53,75.14,66.67,68.93,64.41,63.84,64.97,59.32,57.63,57.06,54.24,54.80,53.11,54.24,50.85,48.59,48.59,48.02,44.63,44.07,46.33,47.46,45.76,50.85,0.55,0.67,0.62,0.68,0.55,0.56,0.61,0.43,0.43,0.40,0.25,0.34,0.22,0.31,0.14,0.00,-0.07,-0.13,-0.15,-0.17,-0.14,-0.19,-0.14,0.09,177,56.710000,0.244167,-2352.330000,43.217500
52,PriceChange_LinearReg,18,8,0,89.72,74.31,66.80,67.19,64.43,63.24,65.61,62.06,57.31,56.52,53.75,54.55,52.57,53.75,51.38,49.80,49.41,49.01,45.45,43.87,45.85,46.64,45.06,48.62,0.50,0.55,0.43,0.57,0.50,0.44,0.45,0.41,0.37,0.34,0.24,0.18,0.10,0.30,0.14,0.00,0.00,0.00,-0.15,-0.15,-0.25,-0.29,-0.27,0.00,253,56.537500,0.183750,-3406.012500,46.488750
58,PriceChange_LinearReg,20,8,0,89.74,73.99,65.57,66.30,64.47,63.00,64.84,61.54,57.14,56.78,54.95,54.58,53.11,54.21,52.01,50.18,49.82,49.45,45.79,43.96,45.42,46.15,44.32,47.62,0.46,0.50,0.42,0.55,0.47,0.44,0.44,0.36,0.37,0.34,0.23,0.18,0.15,0.22,0.15,0.03,0.00,0.00,-0.15,-0.10,-0.25,-0.29,-0.29,-0.22,273,56.455833,0.167083,-3697.557500,45.613750
40,PriceChange_LinearReg,14,8,0,90.31,74.49,67.35,69.39,65.31,64.29,65.82,61.22,58.16,57.14,54.08,54.59,52.04,53.57,50.00,47.45,46.94,47.96,44.39,42.86,44.90,46.43,44.90,49.49,0.53,0.62,0.57,0.63,0.57,0.55,0.53,0.43,0.44,0.41,0.27,0.22,0.12,0.26,0.01,0.00,-0.14,-0.13,-0.16,-0.22,-0.31,-0.36,-0.25,0.00,196,56.378333,0.191250,-2669.846667,37.485000
46,PriceChange_LinearReg,16,8,0,90.00,74.55,65.91,68.64,63.64,62.73,64.55,60.91,57.27,56.36,53.18,53.64,51.82,53.18,50.45,48.18,47.73,48.18,44.09,42.73,45.45,46.36,45.00,49.09,0.50,0.52,0.45,0.57,0.51,0.43,0.44,0.41,0.40,0.36,0.24,0.19,0.07,0.20,0.05,0.00,-0.09,-0.10,-0.16,-0.16,-0.21,-0.30,-0.25,0.00,220,55.985000,0.169583,-3083.300000,37.308333
51,PriceChange_LinearReg,18,6,0,91.11,74.67,67.11,68.00,64.44,61.78,64.44,61.78,57.33,56.44,54.22,54.67,52.89,

In [2]:
# 15m/1h
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
stat.sort_values('pct_right_forecast_avg', ascending=False).head(20)

,pattern,PriceChange_low_price_quantile,LinearReg_timeperiod,LinearReg_low_bound,pct_right_forecast_1,pct_right_forecast_2,pct_right_forecast_3,pct_right_forecast_4,pct_right_forecast_5,pct_right_forecast_6,pct_right_forecast_7,pct_right_forecast_8,pct_right_forecast_9,pct_right_forecast_10,pct_right_forecast_11,pct_right_forecast_12,pct_right_forecast_13,pct_right_forecast_14,pct_right_forecast_15,pct_right_forecast_16,pct_right_forecast_17,pct_right_forecast_18,pct_right_forecast_19,pct_right_forecast_20,pct_right_forecast_21,pct_right_forecast_22,pct_right_forecast_23,pct_right_forecast_24,pct_price_diff_1,pct_price_diff_2,pct_price_diff_3,pct_price_diff_4,pct_price_diff_5,pct_price_diff_6,pct_price_diff_7,pct_price_diff_8,pct_price_diff_9,pct_price_diff_10,pct_price_diff_11,pct_price_diff_12,pct_price_diff_13,pct_price_diff_14,pct_price_diff_15,pct_price_diff_16,pct_price_diff_17,pct_price_diff_18,pct_price_diff_19,pct_price_diff_20,pct_price_diff_21,pct_price_diff_22,pct_price_diff_23,pct_price_diff_24,forecasts_num,pct_right_forecast_avg,pct_price_diff_avg,forecast_rank,price_rank
2,PriceChange_LinearReg,1,10,0,88.89,77.78,75.00,69.44,58.33,61.11,63.89,66.67,61.11,52.78,58.33,61.11,58.33,52.78,52.78,50.00,52.78,52.78,52.78,50.00,50.00,47.22,47.22,47.22,2.86,2.40,2.18,2.00,1.48,2.42,1.21,1.44,1.48,0.57,0.88,1.10,1.41,0.57,0.63,1.12,0.58,0.30,0.30,0.15,0.12,-0.24,-0.62,-0.38,36,58.680417,0.998333,-407.505000,35.940000
58,PriceChange_LinearReg,20,8,0,93.33,71.19,68.33,63.10,60.71,56.90,55.95,57.38,56.19,54.52,55.95,55.00,52.38,53.10,52.86,53.10,50.48,48.81,47.62,45.95,48.33,49.05,49.29,47.38,0.89,0.87,0.61,0.66,0.66,0.53,0.49,0.45,0.40,0.32,0.35,0.36,0.25,0.39,0.35,0.41,0.09,-0.06,-0.05,-0.21,0.00,0.00,0.00,-0.10,420,56.120833,0.319167,-5829.250000,134.050000
55,PriceChange_LinearReg,19,8,0,93.12,71.99,68.80,62.65,60.20,56.27,56.02,57.25,56.02,54.30,55.77,55.04,52.33,52.83,52.58,52.83,49.88,48.40,47.17,45.45,47.91,48.89,49.14,46.93,0.90,0.88,0.61,0.65,0.64,0.51,0.48,0.44,0.37,0.31,0.35,0.33,0.25,0.37,0.32,0.38,0.00,-0.09,-0.16,-0.22,0.00,0.00,0.00,-0.14,407,55.907083,0.299167,-5735.817083,121.760833
52,PriceChange_LinearReg,18,8,0,93.08,72.05,68.97,62.82,60.26,55.64,56.15,57.44,55.38,53.85,55.13,54.87,52.31,52.56,52.31,52.82,49.23,48.21,46.92,45.38,47.18,48.72,49.49,46.92,0.91,0.88,0.61,0.65,0.63,0.50,0.49,0.44,0.32,0.29,0.33,0.32,0.25,0.29,0.30,0.35,-0.01,-0.17,-0.21,-0.31,-0.06,0.00,0.00,-0.14,390,55.737083,0.277500,-5562.537500,108.225000
40,PriceChange_LinearReg,14,8,0,92.31,71.79,68.59,63.46,60.90,57.05,56.41,57.05,54.49,54.17,55.45,54.49,52.24,52.88,51.92,52.24,49.68,48.40,47.44,45.83,46.47,48.08,49.68,45.83,1.01,1.03,0.71,0.74,0.70,0.59,0.53,0.48,0.26,0.31,0.31,0.29,0.24,0.24,0.36,0.32,0.00,-0.26,-0.26,-0.38,-0.24,0.00,0.00,-0.16,312,55.702083,0.284167,-4460.950000,88.660000
57,PriceChange_LinearReg,20,6,0,93.20,71.03,68.26,62.97,61.21,57.18,55.92,57.43,55.92,54.16,55.42,54.41,52.39,53.15,52.39,52.39,49.37,47.86,46.60,45.09,47.10,48.11,48.11,46.10,0.91,0.88,0.61,0.65,0.68,0.56,0.49,0.49,0.37,0.31,0.32,0.32,0.25,0.37,0.32,0.33,0.00,-0.15,-0.19,-0.30,-0.14,0.00,0.00,-0.15,397,55.657083,0.288750,-5694.137917,114.633750
59,PriceChange_LinearReg,20,10,0,93.68,71.33,67.72,63.21,59.82,56.21,54.85,56.43,55.30,54.18,55.53,54.85,51.69,52.37,52.37,52.60,49.89,48.08,46.73,45.37,47.86,48.53,48.98,47.86,0.89,0.83,0.58,0.65,0.63,0.50,0.44,0.43,0.37,0.31,0.35,0.39,0.23,0.26,0.28,0.31,0.00,-0.14,-0.10,-0.22,-0.02,0.00,0.00,-0.09,443,55.643333,0.286667,-6360.003333,126.993333
49,PriceChange_LinearReg,17,8,0,93.26,72.24,68.46,62.26,59.84,55.26,55.80,56.87,54.99,53.64,54.99,54.18,52.29,52.56,52.29,52.56,49.33,47.98,47.17,46.09,47.44,48.52,49.60,46.90,0.95,0.88,0.61,0.66,0.58,0.50,0.48,0.43,0.27,0.29,0.31,0.27,0.24,0.26,0.32,0.33,0.00,-0.17,-0.22,-0.32,-0.02,0.00,0.00,-0.15,371,55.605000,0.270833,-5340.545000,100.479167
13,PriceChange_LinearReg,5,8,0,91.73,72.18,76.69,64.66,60.90,57.89,54.89,56.39,57.14,54.89,57.14,55.64,50.38,51.13,49.6

# Save new config data to config file

In [3]:
from config_updater import ConfigUpdater

ttype = 'buy'
pattern = ['PriceChange', 'LinearReg']
work_timeframe = '15m'
higher_timeframe = '1h'
timeframe = f'{work_timeframe}_{higher_timeframe}'
        
optim_dict = {'PriceChange': {"low_price_quantile": [20]}, 
              'LinearReg': {'timeperiod': [8], 'low_bound': [0]}}
        
cu = ConfigUpdater(ttype, timeframe)
cu.config_update(optim_dict)

# Check global statistics

In [3]:
from glob import glob

ttype = 'buy'
pattern = ['PriceChange']
work_timeframe = '5m'
higher_timeframe = '1h'
stat_list = glob(f'opt_{"_".join(pattern)}_{ttype}_{work_timeframe}_{higher_timeframe}*')
stat = None

for sl in stat_list:
    tmp = pd.read_pickle(sl)
    if stat is None:
        stat = tmp.copy()
    else:
        stat = pd.concat([stat, tmp])
        
stat['pct_right_forecast_avg'] = stat[[f'pct_right_forecast_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['pct_price_diff_avg'] = stat[[f'pct_price_diff_{lag + 1}' for lag in range(24)]].apply(np.mean, axis=1)
stat['forecast_rank'] = (stat['pct_right_forecast_avg'] - 70) * stat['forecasts_num']
stat['price_rank'] = stat['pct_price_diff_avg'] * stat['forecasts_num']
        
total_stat = stat.groupby(['PriceChange_low_price_quantile']).agg({'pct_right_forecast_avg': 'mean',
                                                                   'pct_price_diff_avg': 'mean',
                                                                   'forecast_rank': 'mean', 
                                                                   'price_rank': 'mean', 
                                                                   'forecasts_num': 'sum'}).sort_values('forecast_rank', 
                                                                                                        ascending=False)
total_stat.head(20)

,pct_right_forecast_avg,pct_price_diff_avg,forecast_rank,price_rank,forecasts_num
PriceChange_low_price_quantile,,,,,
1,48.911181,0.698576,-44.928403,0.749826,33
2,53.224514,1.224826,-96.531319,0.555521,60
4,63.002222,1.507500,-106.731771,3.426736,123
3,57.661771,1.482188,-108.683125,1.100035,90
5,63.067674,1.100035,-111.523819,7.445278,154
6,63.261736,1.080729,-129.090833,7.653299,176
7,59.737639,0.319965,-155.473542,7.016875,216
8,60.148194,0.825590,-160.278958,10.196632,242
9,60.661910,0.429826,-167.830035,11.735139,278
